# 色んなモデルを実装していく

In [1]:
import torch
import torch.nn.functional as F
import numpy as np
from glob import glob
import cv2

In [2]:
num_classes = 2
img_height, img_width = 32, 32
channel = 3
GPU = False
torch.manual_seed(0)

In [3]:
def data_load():
    xs = np.ndarray((0, img_height, img_width, 3))
    ts = np.ndarray((0))
    
    for dir_path in glob('../Dataset/train/images/*'):
        for path in glob(dir_path + '/*'):
            x = cv2.imread(path)
            x = cv2.resize(x, (img_width, img_height)).astype(np.float32)
            x /= 255.
            xs = np.r_[xs, x[None, ...]]

            t = np.zeros((1))
            if 'akahara' in path:
                t = np.array((0))
            elif 'madara' in path:
                t = np.array((1))
            ts = np.r_[ts, t]

    xs = xs.transpose(0,3,1,2)

    return xs, ts

In [4]:
def train():
    device = torch.device("cuda" if GPU else "cpu")
    
    #model
    model = Mynet().to(device)
    opt = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    
    #トレーニングモードに変更
    model.train()
    
    xs, ts = data_load()
    
    #training
    mb = 8
    mbi = 0
    train_ind = np.arange(len(xs))
    np.random.seed(0)
    np.random.shuffle(train_ind)
    
    for i in range(100):
        if mbi + mb > len(xs):
            mb_ind = train_ind[mbi:]
            np.random.shuffle(train_ind)
            mb_ind = np.hstack((mb_ind, train_ind[:(mb-(len(xs)-mbi))]))
        else:
            mb_ind = train_ind[mbi:mbi+mb]
            mbi += mb
            
        x = torch.tensor(xs[mb_ind], dtype=torch.float).to(device)
        t = torch.tensor(ts[mb_ind], dtype=torch.long).to(device)
        
        opt.zero_grad()
        y = model(x)
        y = F.log_softmax(y, dim=1)
        loss = torch.nn.CrossEntropyLoss()(y, t)
        loss.backward()
        opt.step()
        
        pred = y.argmax(dim=1, keepdim=True)
        acc = pred.eq(t.view_as(pred)).sum().item() / mb
        
        print("iter >>", i+1, ',loss >>', loss.item(), ',accuracy >>', acc)
        
    torch.save(model.state_dict(), "cnn.pt")

In [5]:
def test():
    device = torch.device("cuda" if GPU else "cpu")
    model = Mynet().to(device)
    model.eval()
    model.load_state_dict(torch.load("cnn.pt"))
    
    for dir_path in glob('../Dataset/test/images/*'):
        for path in glob(dir_path + '/*'):
            x = cv2.imread(path)
            x = cv2.resize(x, (img_width, img_height)).astype(np.float32)
            x /= 255.
            x = x.transpose(2, 0, 1)
            x = np.expand_dims(x, axis=0)
            x = torch.tensor(x, dtype=torch.float).to(device)
            
            pred = model(x)
            pred = F.softmax(pred, dim=1).detach().cpu().numpy()[0]
            
            print("in {}, predicted probabilities >> {}".format(path, pred))

#### トレーニング，テスト時はモデル名の書いてある部分を全てMynetに変えれば動きます

## LeNet

In [6]:
class LeNet(torch.nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(channel, 6, kernel_size=5, padding=0)
        self.conv2 = torch.nn.Conv2d(6, 16, kernel_size=5, padding=0)
        self.fc1 = torch.nn.Linear(5*5*16, 120)
        self.fc2 = torch.nn.Linear(120, 64)
        self.fc_out = torch.nn.Linear(64, 2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.sigmoid(F.max_pool2d(x, 2))
        x = self.conv2(x)
        x = F.sigmoid(F.max_pool2d(x, 2))
        x = x.view(-1, 5*5*16)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc_out(x)
        x = F.softmax(x, dim=1)
        
        return x

## AlexNet

In [9]:
class AlexNet(torch.nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(channel, 96, kernel_size=11, stride=4, padding=0)
        self.conv2 = torch.nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=0)
        self.conv3 = torch.nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1)
        self.conv4 = torch.nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1)
        self.conv5 = torch.nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1)
        self.fc1 = torch.nn.Linear(6*6*256, 4096)
        self.fc2 = torch.nn.Linear(4096, 4096)
        self.fc_out = torch.nn.Linear(4096, 2)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = torch.nn.modules.normalization.LocalResposeNorm(size=1)(x)
        x = F.max_pool2d(x, 3, stride=2)
        x = F.relu(self.conv2(x))
        x = torch.nn.modules.normalization.LocalResponseNorm(size=1)(x)
        x = F.max_pool2d(x, 3, stride=2)
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = x.view(-1, 6*6*256)
        x = F.relu(self.fc1(x))
        x = torch.nn.Dropout()(x)
        x = F.relu(self.fc2(x))
        x = torch.nn.Dropout()(x)
        x = self.fc_out(x)
        x = F.softmax(x, dim=1)
        return x

## GAN

In [10]:
class Generator(torch.nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = torch.nn.Linear(100, 128)
        self.fc2 = torch.nn.Linear(128, 256)
        self.fc3 = torch.nn.Linear(256, 512)
        self.fc_out = torch.nn.Linear(512, img_width*img_height*channel)
        
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.sigmoid(self.fc1(x))
        return x
    
class Discriminator(torch.nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.l1 = torch.nn.Linear(img_height * img_width * channel, 512)
        self.l2 = torch.nn.Linear(512, 256)
        self.l3 = torch.nn.Linear(256, 1)

    def forward(self, x):
        x = self.l1(x)
        x = torch.nn.functional.leaky_relu(x, 0.2)
        x = self.l2(x)
        x = torch.nn.functional.leaky_relu(x, 0.2)
        x = self.l3(x)
        x = torch.nn.functional.sigmoid(x)
        return x
    